## Marketing insight in Burger King / McDonalds locations

#### Version
- 03-08-2019 /  Léon Smiers, Consolidated version 

#### Introduction
As part of the Marketing class as part of the __[RSM OneMBA](https://www.rsm.nl/mba/global-executive-onemba)__ curriculum I investigate the marketing mix 4P's (Product, Price, Promotion and Place) for Burger King. Here I specifically look at the PLACE aspect of marketing. Using Google APIs and Random Forests delivers insight in what features determine the choice of a Burger King for a specific location  in The Netherlands. 

#### This Notebook
In this notebook I seach for the Burger King locations in The Netherlands including the below mentioned Features. __[Google Place APIs](https://developers.google.com/places/web-service/search#find-place-examples )__ are used to collect the data.
This notebook relates to the "Investigate BK Data.ipynb" Jupyter notebook.

#### WARNING
<font color='red'> **The Google APIs are NOT free of usage**. </font>  See the __[Google APIs Pricing sheet](https://cloud.google.com/maps-platform/pricing/sheet/ )__ for the latest pricing. 

<font color='red'> **The Google Cloud Dashboard billing information DOES NOT PROVIDE real time insight in the usage**.</font> I found out the hard way there is a lag of (at least) several hours. Officially it even takes 48 hours to settle the bill. 

At the current data the pricing was $17 for 1000 Place API calls.
In the below code for each Burger King store ±15 calls are executed. In total there are (at current date) 66 Burger King locations in The Netherlands. In total every Burger King location results into ±1000 API calls.

#### Features 
For the "Place" I used Google APIs to find Burger King (and McDonalds) location including 13 features 
- Shopping nearby
  - Shopping Mall / Shops 
- Travel proximity
  - Subway station  / Train station / Metro / Airport / Gas station 
- Restaurant proximity
  - McD / KFC / Subway / Five Guys
  - Other restaurants nearby (amount)
- All proximity checks are within 500 meter
- One exception gas station within 300 meters

#### Install required
- pip install googlemaps
- pip install -U google-api-python-client

In [2]:
import googlemaps
import pandas as pd
import csv
import logging
import logging
logging.basicConfig(level=logging.INFO)

In [2]:
#Create Google API client 
gmaps = googlemaps.Client(key='[YOUR GOOGLE API KEY]')

In [3]:
def add_dataframes(df_orig, df_new):
    """ add two dataframes, remove double place_ids
    Input : df_orig Dataframe, df_new Dataframe
    Output : df_orig Dataframe
    Used libs : Pandas DataFrame   """    
    
    for index_new, row_new in df_new.iterrows():
        #print("index_new {0}, row_new {1}".format(index_new, row_new))
        #print(row_new.get('place_id'))
        add_row = True
        for index_orig, row_orig in df_orig.iterrows():
            #print('----')
            #print("place id, new {0}, orig {1}".format(row_new.get('place_id'), row_orig.get('place_id')))
            if row_new.get('place_id') == row_orig.get('place_id'):
                #print("gelijk aan elkaar")
                add_row=False
                break
        if add_row:
            #print(type(row_new))
            #print(row_new.values)
            #df_orig = df_orig.assign(e=pd.Series(row_new.values))
            df_orig=df_orig.append(row_new)
            #print(df_orig.append(row_new))
            #print("appended")
            #print(df_orig)
                
    #print(df_orig)
    return df_orig 

In [4]:
def check_resulset(resultset):
    """ Check if resultset returns results
    Input : resultset
    Output : 1 (Yes) when available, 0 (No) when zero results
    Used libs : -  """       

    if resultset.get('status') =='ZERO_RESULTS':
        available =0
    else:
        available = 1
    return available

In [5]:
def determine_amount_locations(resultset):
    """ Check if resultset returns results
    Input : resultset
    Output : range of locations nearby ['0,'1-5','6-10','11-15,'16-20,>20]
            Changed to return actual number with max of 20
    Used libs : -  """       

    amount='0'
    if resultset.get('next_page_token') is None:
        #print('<20')
        no_locations=len(resultset.get('results'))
        
        if no_locations==0:
            amount='[0]'
        elif no_locations<6:
            amount='[1-5]'
        elif no_locations<11:
            amount='[6-10]'
        elif no_locations<16:
            amount='[11-15]'
        else:
            amount='[15-20]'
    else:
        no_locations=20
        amount='[>20]'
    return no_locations
#    return amount


In [6]:
def handle_bk_detail(bk_place_id, bk_dict):
    """ Extract BK detail features from PLACE API
    Input : place_id string, bk_dict dict
    Output : bk_dict dict
    Example structure of address_components in result part, this is a list
    +++++++++
    {'long_name': '333', 'short_name': '333', 'types': ['street_number']}
    +++++++++
    {'long_name': 'Watermanweg', 'short_name': 'Watermanweg', 'types': ['route']}
    +++++++++
    {'long_name': 'Prins Alexander', 'short_name': 'Prins Alexander', 'types': ['sublocality_level_1', 'sublocality', 'political']}
    +++++++++
    {'long_name': 'Rotterdam', 'short_name': 'Rotterdam', 'types': ['locality', 'political']}
    +++++++++
    {'long_name': 'Rotterdam', 'short_name': 'Rotterdam', 'types': ['administrative_area_level_2', 'political']}
    +++++++++
    {'long_name': 'Zuid-Holland', 'short_name': 'ZH', 'types': ['administrative_area_level_1', 'political']}
    +++++++++
    {'long_name': 'Netherlands', 'short_name': 'NL', 'types': ['country', 'political']}
    +++++++++
    {'long_name': '3067 GA', 'short_name': '3067 GA', 'types': ['postal_code']}
    City/Location: Rotterdam, Prins Alexander
    {'location': {'lat': 51.9514875, 'lng': 4.557968}, 'viewport': {'northeast': {'lat': 51.95247203029149, 'lng': 4.55865995}, 'southwest': {'lat': 51.9497740697085, 'lng': 4.555892149999998}}}

    Possible values
    ['street_number']
    ['route']
    ['sublocality_level_1', 'sublocality', 'political']
    ['locality', 'political']
    ['administrative_area_level_2', 'political']
    ['administrative_area_level_1', 'political']
    ['country', 'political']
    ['postal_code']

    Used libs : googlemaps.Client place  """  

    ####################
    ## ADDRESS
    ####################    
    bk_detail = gmaps.place(place_id=bk_place_id)      #Get place resultset
    bk_detail_result=bk_detail.get('result')           #Get result part
    address=bk_detail_result.get('address_components') #Get address part
    
    street_number=''
    route=''
    location=''
    city=''
    postal_code=''
    area=''
    city=''  
    province=''
    country=''
    
    for add in address:
        types=add.get('types')
        #print("TYPES {0}, values {1}, type0types {2}".format(types,add.get('long_name'),type(types)))
        if types[0] == 'street_number':
            street_number=add.get('long_name')
            continue
        
        if types[0] == 'route':
            route=add.get('long_name')
            continue

        if types[0] == 'postal_code':
            postal_code=add.get('long_name')
            continue

        if types[0] == 'administrative_area_level_1':
            province=add.get('long_name')
            continue

        if types[0] == 'country':
            country=add.get('long_name')
            continue                        
            
        #Area
        check_locality=False
        for t in types:
            if t == 'locality':
                check_locality=True
            if check_locality:
                area=add.get('long_name')
            continue

        #City
        check_city=False
        for t in types:
            if t == 'administrative_area_level_2':
                check_city=True
            if check_city:
                city=add.get('long_name')

    ####################
    ## GEOMETRY
    ####################   
    location_lat=''
    location_lng=''    
    #{'location': {'lat': 51.9315951, 'lng': 4.5890279}, 'viewport': {'northeast': {'lat': 51.9330918302915, 'lng': 4.590852930291502}, 'southwest': {'lat': 51.9303938697085, 'lng': 4.588154969708498}}}
    geo=bk_detail_result.get('geometry')
    location=geo.get('location')
    location_lat=location.get('lat')
    location_lng=location.get('lng')
            
#    if country == 'Netherlands':       
    bk_dict['street_no'] = route + " " + street_number
    bk_dict['area'] = area
    bk_dict['city'] = city
    bk_dict['postal_code'] = postal_code
    bk_dict['location_lat'] = location_lat
    bk_dict['location_lng'] = location_lng
    bk_dict['province'] = province
    bk_dict['country'] = country
    
    return bk_dict

In [7]:
def handle_places_nearby(bk_dict, lat, lng ):
    """ Extract nearest places to a specific BK 
    Input : bk_result dict, [lat, lng] of BK
    Output : bk dict
    Used libs : googlemaps.Client places_nearby   """       

    #print("lat: {0} lng {1}".format(lat, lng))
    
    _radius = '500' #Radius of locations around Burger King
    _radius_gas_station = '300' #Radius of gas station to Burger King

    ##########################
    #Check nearby restaurants
    #########################
    amount_rest='0'
    bk_nearest_restaurants = gmaps.places_nearby(location={'lat': lat, 'lng': lng}, radius = _radius,type='restaurant')     
    amount_rest = determine_amount_locations(bk_nearest_restaurants)
    bk_dict['number_of_closeby_restaurants'] = amount_rest
    
    ##########################
    #Check nearby meal_takeaway
    #########################
    amount_rest='0'
    bk_nearest_meal_takeaway = gmaps.places_nearby(location={'lat': lat, 'lng': lng}, radius = _radius,type='meal_takeaway')     
    amount_meal_takeaway = determine_amount_locations(bk_nearest_meal_takeaway)
    bk_dict['number_of_closeby_takeaways'] = amount_meal_takeaway

    ########################################
    #Check nearby brands McD, KFC, Five Guys
    ########################################
    bk_nearest_bk = gmaps.places_nearby(location={'lat': lat, 'lng': lng}, radius = _radius, name='burger king')     
    bk_nearest_mcd = gmaps.places_nearby(location={'lat': lat, 'lng': lng}, radius = _radius, name='McDonald\'s')     
    bk_nearest_kfc = gmaps.places_nearby(location={'lat': lat, 'lng': lng}, radius = _radius, name='KFC')     
    bk_nearest_fiveguys = gmaps.places_nearby(location={'lat': lat, 'lng': lng}, radius = _radius, name='Five Guys')     
    bk_nearest_subway = gmaps.places_nearby(location={'lat': lat, 'lng': lng}, radius = _radius, name='Subway')     

    bk_dict['BK_nearby'] = check_resulset(bk_nearest_bk)
    bk_dict['McDonalds_nearby'] = check_resulset(bk_nearest_mcd)
    bk_dict['KFC_nearby'] = check_resulset(bk_nearest_kfc)
    bk_dict['Five_Guys_nearby'] = check_resulset(bk_nearest_fiveguys)
    bk_dict['Subway_nearby'] = check_resulset(bk_nearest_subway)
    
    ########################################
    #Check nearby shopping mall / stores 
    ########################################
    bk_nearest_shopping_mall = gmaps.places_nearby(location={'lat': lat, 'lng': lng}, radius = _radius, type='shopping_mall')     
    bk_dict['shopping_mall_nearby'] = check_resulset(bk_nearest_shopping_mall)

    bk_nearest_store = gmaps.places_nearby(location={'lat': lat, 'lng': lng}, radius = _radius,type='store')     
    amount_store = determine_amount_locations(bk_nearest_store)
    bk_dict['number_of_closeby_stores'] = amount_store

    ########################################
    #Check nearby airport / subway_station / train_station / gas_station
    ########################################
    bk_nearest_airport = gmaps.places_nearby(location={'lat': lat, 'lng': lng}, radius = _radius, type='airport')     
    bk_nearest_subway_station = gmaps.places_nearby(location={'lat': lat, 'lng': lng}, radius = _radius, type='subway_station')     
    bk_nearest_train_station  = gmaps.places_nearby(location={'lat': lat, 'lng': lng}, radius = _radius, type='train_station')     
    bk_nearest_gas_station  = gmaps.places_nearby(location={'lat': lat, 'lng': lng}, radius = _radius_gas_station, type='gas_station')     

    bk_dict['airport_nearby'] = check_resulset(bk_nearest_airport)
    bk_dict['subway_station_nearby'] = check_resulset(bk_nearest_subway_station)
    bk_dict['train_station_nearby'] = check_resulset(bk_nearest_train_station)
    bk_dict['gas_station_nearby'] = check_resulset(bk_nearest_gas_station)
    
    return bk_dict    

In [8]:
def handle_bk_details(bk_result):
    """ Extract BK features from PLACES API
    Input : bk_result dict
    Output : bk dict
    Used libs : googlemaps.Client places   """       
    bk={} #empty bk dict
    place_id = bk_result.get('place_id')
    bk['place_id']=place_id

    #Place details (address, location)
    bk=handle_bk_detail(place_id, bk)
    
    #print(bk)
    return bk

In [9]:
def handle_main_query(query_statement, company):
    """ Google API delivers 20 results per query, with a maximum of 60
        This calls for some workaround, instead of running the query on country level, run this on province level
        this main query function executes the query and delivers a dictionary
    Input : query, admin_area
    Output : bk dict
    Used libs : googlemaps.Client places   """       

    bk_list=[]
    handle_resultset=True
    first_query=True
    counter=1

    logging.info('--------------------------------')
    logging.info('-- Query: {0}'.format(query_statement))
    
    count=0
    while handle_resultset:
        logging.info("--  Query batch (20 each time): {0}".format(counter))
        counter=counter+1
        if first_query:
            bks=gmaps.places(query=query_statement)
            first_query=False
        else:
            bks=gmaps.places(query=query_statement,page_token=next_page_token,type='')
        next_page_token = bks.get('next_page_token') #Google API Places return maximum 20 rows, with token next resultset can be obtained
        #print("next_page_token {0}".format(next_page_token))
    
        bk_results=bks.get('results') #Obtain the resultset to work against

        #print("Keys in place API result:")
        #for bkr in bk_results[0]:
        #    print("- {0}".format(bkr))
        
        i=1
        for bkr in bk_results:
            count=count+1
            logging.info("--  Query detail count: {0}".format(i))
            i=i+1
            bk_dict=handle_bk_details(bkr)   #get details of BK
            bk_dict["company"]=company
            
#            print("len(bk_dict) {0}".format(len(bk_dict))) ##############
            if bk_dict.get('country')=='Netherlands':
                bk_dict=handle_places_nearby(bk_dict,bk_dict.get('location_lat'),bk_dict.get('location_lng'))
                bk_list.append(bk_dict)
            #break

        if next_page_token is None:
            handle_resultset=False
    logging.info("--Total for query: {0}".format(count))
    logging.info('--------------------------------')
            
    return bk_list

In [12]:
def run_company(company):
    """ Run for one company (BK, McD, KFC, Subway)
    Input : company name
    Output : bk_list_company list, df_company DataFrame
    Used libs : -   """    
    
#    provincies=['gelderland']
    provincies=['friesland','groningen','drenthe',
                'overijssel','gelderland','utrecht',
                'Zuid-Holland','Noord-Holland','Noord-Brabant',
                'limburg','zeeland'
               ]

    country="nederland"
    bk_list_company = []
    df_company = pd.DataFrame() 
    
    for prv in provincies:
        query_statement =company + "+" + prv + "+" + country
        print('Query: {0}'.format(query_statement))
        bk_list=handle_main_query(query_statement, company)

        df_province=pd.DataFrame(bk_list) 
        df_company=add_dataframes(df_company,df_province)
        
    return df_company

In [13]:
####################
## MAIN
####################
bk_list_main=[]

# Calling DataFrame constructor on list 
df = pd.DataFrame() 

#companies=['burger+king','McDonald\'s','KFC','Subway']
#companies=['burger+king','McDonald\'s']
companies=['burger+king']
for comp in companies:
    df=df.append(run_company(comp))
    
for col in df.columns: 
    print(col) 
#frame = frame[['second thing', 'other thing', 'one thing']]
df=df[['company',
       'street_no', 'area', 'city', 'postal_code', 'province', 'country',
       'number_of_closeby_restaurants', 'number_of_closeby_takeaways',
       'BK_nearby','Five_Guys_nearby','KFC_nearby','McDonalds_nearby','Subway_nearby',
       'shopping_mall_nearby',
       'airport_nearby','gas_station_nearby','subway_station_nearby','train_station_nearby',
       'number_of_closeby_stores',
       'place_id','location_lat','location_lng']]
df
#### CHANGE BELOW NAME TO YOUR NAMING
df.to_csv("burger_king.csv", sep=',', na_rep='empty',index_label=False, index=False)

INFO:root:--------------------------------
INFO:root:-- Query: burger+king+friesland+nederland
INFO:root:--  Query batch (20 each time): 1


Query: burger+king+friesland+nederland


INFO:root:--  Query detail count: 1
INFO:root:--  Query detail count: 2
INFO:root:--  Query detail count: 3
INFO:root:--  Query detail count: 4
INFO:root:--  Query detail count: 5
INFO:root:--Total for query: 5
INFO:root:--------------------------------
INFO:root:--------------------------------
INFO:root:-- Query: burger+king+groningen+nederland
INFO:root:--  Query batch (20 each time): 1


Query: burger+king+groningen+nederland


INFO:root:--  Query detail count: 1
INFO:root:--  Query detail count: 2
INFO:root:--Total for query: 2
INFO:root:--------------------------------
INFO:root:--------------------------------
INFO:root:-- Query: burger+king+drenthe+nederland
INFO:root:--  Query batch (20 each time): 1


...

Query: burger+king+limburg+nederland


INFO:root:--  Query detail count: 1
INFO:root:--  Query detail count: 2
INFO:root:--  Query detail count: 3
INFO:root:--  Query detail count: 4
INFO:root:--  Query detail count: 5
INFO:root:--Total for query: 5
INFO:root:--------------------------------
INFO:root:--------------------------------
INFO:root:-- Query: burger+king+zeeland+nederland
INFO:root:--  Query batch (20 each time): 1


Query: burger+king+zeeland+nederland


INFO:root:--  Query detail count: 1
INFO:root:--Total for query: 1
INFO:root:--------------------------------


BK_nearby
Five_Guys_nearby
KFC_nearby
McDonalds_nearby
Subway_nearby
airport_nearby
area
city
company
country
gas_station_nearby
location_lat
location_lng
number_of_closeby_restaurants
number_of_closeby_stores
number_of_closeby_takeaways
place_id
postal_code
province
shopping_mall_nearby
street_no
subway_station_nearby
train_station_nearby
